# Yandex.Music

Comparison of Moscow and St. Petersburg is surrounded by myths. For example:
 * Moscow is a metropolis subject to the rigid rhythm of the working week;
 * St. Petersburg is a cultural capital, with its own tastes.

Using Yandex.Music data, you will compare the behavior of users in the two capitals.

**Research Purpose** - Test three hypotheses:
1. User activity depends on the day of the week. Moreover, in Moscow and St. Petersburg this manifests itself in different ways.
2. On Monday morning, certain genres prevail in Moscow, while others prevail in St. Petersburg. Similarly, Friday evenings are dominated by different genres, depending on the city.
3. Moscow and St. Petersburg prefer different genres of music. In Moscow, they listen to pop music more often, in St. Petersburg - Russian rap.

**Research Progress**

You will receive data on user behavior from the `yandex_music_project.csv` file. Nothing is known about the quality of the data. Therefore, before testing hypotheses, a review of the data is required.

You will check the data for errors and assess their impact on the study. Then, in the pre-processing phase, you will look for opportunities to correct the most critical data errors.
 
Thus, the study will take place in three stages:
 1. Data review.
 2. Data preprocessing.
 3. Hypothesis testing.



## Data review






Основной инструмент аналитика — `pandas`. Импортируйте эту библиотеку.

In [1]:
# pandas library import
import pandas as pd

Прочитайте файл `yandex_music_project.csv` из папки `/datasets` и сохраните его в переменной `df`:

In [2]:
# reading data file and saving to df
df = pd.read_csv("/datasets/" "yandex_music_project.csv")




Выведите на экран первые десять строк таблицы:

In [3]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Одной командой получить общую информацию о таблице:

In [4]:
# getting general information about the data in the df table
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
  userID    65079 non-null object
Track       63848 non-null object
artist      57876 non-null object
genre       63881 non-null object
  City      65079 non-null object
time        65079 non-null object
Day         65079 non-null object
dtypes: object(7)
memory usage: 3.5+ MB


So, the table has seven columns. The data type in all columns is `object`.

According to the data documentation:
* `userID`
* `Track` 
* `artist` 
* `genre` 
* `City` 
* `time` 
* `Day` 

There are three style violations in the column headings:
1. Lowercase letters are combined with uppercase.
2. There are gaps.
3. In the "userID" column, it's better to do "user_id" for good style.



The number of values ​​in the columns varies. This means there are missing values ​​in the data.


**Сonclusion**

Each line of the table contains data about the track you have listened to. Some of the columns describe the composition itself: title, artist and genre. The rest of the data tells about the user: what city he is from, when he listened to music.

Preliminarily, it can be argued that there is enough data to test hypotheses. But there are gaps in the data, and discrepancies in the names of the columns with good style.

To move forward, you need to fix problems in the data.

## Data preprocessing 
Let's correct the style in the column headings, eliminate gaps. Then check the data for duplicates.

### Heading style


In [5]:
# list of column names of table df
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Let's bring the names in line with good style:
* write a few words in the title in "snake_register",
* make all characters lowercase,
* eliminate spaces.

To do this, rename the columns like this:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [6]:
# renaming columns
df = df.rename(columns = {"  userID" :"user_id", "Track" : "track",   '  City  ': "city", "Day" : "day" })

Проверьте результат. Для этого ещё раз выведите на экран названия столбцов:

In [7]:
# check results - list column names
df.head()

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


### Missing values
First, let's calculate how many missing values ​​are in the table. There are two methods for this. `pandas`:

In [8]:
# skip count
print(df.isna().sum())

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64


Not all missing values ​​affect the study. So in `track` and `artist` the gaps are not important for your work. It suffices to replace them with explicit notation.

But omissions in `genre` can interfere with the comparison of musical tastes in Moscow and St. Petersburg. In practice, it would be correct to determine the cause of the gaps and restore the data. This option is not available in the curriculum. Have to:
* fill in these gaps with explicit notation,
* estimate how much they will damage the calculations.


Replace the missing values ​​in the `track`, `artist` and `genre` columns with the string `'unknown'`. To do this, create a `columns_to_replace` list, iterate through its elements with a `for` loop, and for each column, replace the missing values:

In [9]:
# loop through column names and replace missing values ​​with 'unknown'
# I don't see much point in doing a loop, because you can just create a list, and apply the fillna list, the result will be the same.
columns_to_replace = ["track", "artist" , "genre"]
df[columns_to_replace] = df[columns_to_replace].fillna("unknown")

Make sure there are no gaps in the table. To do this, count the missing values ​​again.

In [10]:
# skip count
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Duplicates
Let's count explicit duplicates in the table with one command:

In [11]:
# counting clear duplicates
df.duplicated().sum()

3826

Call the special `pandas` method to remove obvious duplicates:

In [12]:
# removal of explicit duplicates (with the removal of old indexes and the formation of new ones)
df = df.drop_duplicates().reset_index(drop=True)

Once again, let's count the obvious duplicates in the table - make sure you get rid of them completely:

In [ ]:
# check for duplicates
df.duplicated().sum()


Now let's get rid of the implicit duplicates in the `genre` column. For example, the name of the same genre can be spelled slightly differently. Such errors will also affect the result of the study.

Let's display a list of unique genre names, sorted alphabetically. For this:
* extract the desired dataframe column,
* apply a sort method to it,
* for a sorted column, call a method that will return the unique values ​​from the column.

In [14]:
# View Unique Genre Names
df["genre"].unique()


array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb', 'hip',
       'jazz', 'postrock', 'latin', 'classical', 'metal', 'reggae',
       'tatar', 'blues', 'instrumental', 'rusrock', 'dnb', 'türk', 'post',
       'country', 'psychedelic', 'conjazz', 'indie', 'posthardcore',
       'local', 'avantgarde', 'punk', 'videogame', 'techno', 'house',
       'christmas', 'melodic', 'caucasian', 'reggaeton', 'soundtrack',
       'singer', 'ska', 'shanson', 'ambient', 'film', 'western', 'rap',
       'beats', "hard'n'heavy", 'progmetal', 'minimal', 'contemporary',
       'new', 'soul', 'holiday', 'german', 'tropical', 'fairytail',
       'spiritual', 'urban', 'gospel', 'nujazz', 'folkmetal', 'trance',
       'miscellaneous', 'anime', 'hardcore', 'progressive', 'chanson',
       'numetal', 'vocal', 'estrada', 'russian', 'classicmetal',
       'dubstep', 'club', 'deep', 'southern', 'black', 'folkrock',
       'fitness', '

Let's go through the list and look for implicit duplicates of the name `hiphop`. These may be misspelled titles or alternative titles in the same genre.

You will see the following implicit duplicates:
**hip*,
**hop*,
**hip-hop*.

To clear the table of them, write the `replace_wrong_genres()` function with two parameters:
* `wrong_genres` - list of duplicates,
* `correct_genre` is a string with the correct value.

The function should correct the `genre` column in the `df` table: replace each value from the `wrong_genres` list with a value from `correct_genre`.

In [15]:
# Function to replace implicit duplicates
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df["genre"] =df["genre"].replace(wrong_genre, correct_genre)

Let's call `replace_wrong_genres()` and pass it arguments such that it eliminates implicit duplicates: instead of `hip`, `hop` and `hip-hop` the table should have the value `hiphop`:

In [16]:
# Eliminate Implicit Duplicates
wr_d = ["hip", "hop", "hip-hop"]
correct_genre = "hiphop"
replace_wrong_genres(wr_d, correct_genre)

Let's check that we have replaced the wrong names:

* hip
* hop
* hip-hop

Print a sorted list of unique values ​​in the `genre` column:

In [17]:
# Checking for Implicit Duplicates
df["genre"].unique()

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb',
       'hiphop', 'jazz', 'postrock', 'latin', 'classical', 'metal',
       'reggae', 'tatar', 'blues', 'instrumental', 'rusrock', 'dnb',
       'türk', 'post', 'country', 'psychedelic', 'conjazz', 'indie',
       'posthardcore', 'local', 'avantgarde', 'punk', 'videogame',
       'techno', 'house', 'christmas', 'melodic', 'caucasian',
       'reggaeton', 'soundtrack', 'singer', 'ska', 'shanson', 'ambient',
       'film', 'western', 'rap', 'beats', "hard'n'heavy", 'progmetal',
       'minimal', 'contemporary', 'new', 'soul', 'holiday', 'german',
       'tropical', 'fairytail', 'spiritual', 'urban', 'gospel', 'nujazz',
       'folkmetal', 'trance', 'miscellaneous', 'anime', 'hardcore',
       'progressive', 'chanson', 'numetal', 'vocal', 'estrada', 'russian',
       'classicmetal', 'dubstep', 'club', 'deep', 'southern', 'black',
       'folkrock', 'fitness'

**Findings**

Preprocessing found three problems in the data:

- headline style violations,
- missing values,
- duplicates - explicit and implicit.

You've fixed the headers to make the table easier to work with. Without duplicates, the study will become more accurate.

You have replaced missing values ​​with `'unknown'`. It remains to be seen whether the gaps in the `genre` column will harm the study.

Now we can move on to hypothesis testing.

## Hypothesis testing

### Comparison of user behavior of two capitals

The first hypothesis states that users listen to music differently in Moscow and St. Petersburg. Check this assumption against the data on the three days of the week - Monday, Wednesday and Friday. For this:

* Separate users of Moscow and St. Petersburg
* Compare how many tracks each user group listened to on Monday, Wednesday and Friday.


For training, first perform each of the calculations separately.

Estimate user activity in each city. Group the data by city and count the plays in each group.


In [18]:
# Counting plays in each city
c = df.groupby("city").count()
display(c)
                             



,user_id,track,artist,genre,time,day
city,,,,,,
Moscow,42741,42741,42741,42741,42741,42741
Saint-Petersburg,18512,18512,18512,18512,18512,18512


There are more auditions in Moscow than in St. Petersburg. It does not follow from this that Moscow users listen to music more often. There are simply more users in Moscow.

Now group the data by day of the week and count the plays on Monday, Wednesday, and Friday. Please note that the data only contains information about the plays for these days only.



In [19]:
# Play count for each of the three days
d = df.groupby("day")["city"].count()
display(d)

day
Friday       21840
Monday       21354
Wednesday    18059
Name: city, dtype: int64

On average, users from the two cities are less active on Wednesdays. But the picture may change if we consider each city separately.

You have seen how grouping by city and by day of the week works. Now write a function that combines these two calculations.

Create a `number_tracks()` function that will count the plays for a given day and city. She needs two parameters:
* day of the week,
* city name.

In the function, save to a variable the rows of the source table that have the value:
  * in the `day` column is equal to the `day` parameter,
  * in the `city` column is equal to the `city` parameter.

To do this, apply sequential filtering with logical indexing.

Then count the values ​​in the `user_id` column of the resulting table. Save the result to a new variable. Return this variable from the function.

In [20]:
# <create number_tracks() function>
# A function is declared with two parameters: day, city.
# The track_list variable stores those rows of the df table for which
# the value in the 'day' column is equal to the day parameter and at the same time the value
# in the 'city' column is equal to the city parameter (use sequential filtering
# using logical indexing).
# The track_list_count variable stores the number of values ​​in the 'user_id' column,
# calculated by the count() method for the track_list table.
# The function returns a number - the value of track_list_count.

# Function for counting plays for a specific city and day.
# Using sequential filtering with logical indexing, it
# first get the rows with the desired day from the source table,
# then from the result it will filter the rows with the desired city,
# the count() method will count the number of values ​​in the user_id column.
# This is the number the function will return as a result
def number_tracks(day, city):
    track_list = df[(df["day"] == day) & (df["city"] == city)]
    track_list_count = track_list["user_id"].count()
    return track_list_count
        
    

Let's call `number_tracks()` six times, changing the value of the parameters so that we get data for each city on each of the three days.

In [21]:
# number of auditions in Moscow on Mondays
number_tracks( "Monday", "Moscow")

15740

In [22]:

number_tracks( "Monday", "Saint-Petersburg")

5614

In [23]:

number_tracks( "Wednesday", "Moscow")

11056

In [24]:

number_tracks( "Wednesday", "Saint-Petersburg")

7003

In [25]:

number_tracks( "Friday", "Moscow")

15945

In [26]:

number_tracks( "Friday", "Saint-Petersburg")

5895

Create a table using the `pd.DataFrame` constructor, where
* column names - `['city', 'monday', 'wednesday', 'friday']`;
* data is the results you got with `number_tracks`.

In [27]:
# Table with results
columns = ['city', 'monday', 'wednesday', 'friday']
data = [["Moscow", 15740, 11056, 15945],
        ["Saint-Petersburg",5614, 7003, 5895]]
info = pd.DataFrame(data=data, columns = columns)
display(info)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Findings**

The data shows the difference in user behavior:

- In Moscow, the peak of listening falls on Monday and Friday, and on Wednesday there is a noticeable decline.
- In St. Petersburg, on the contrary, they listen to music more on Wednesdays. Activity on Monday and Friday here is almost equally inferior to Wednesday.

So the data support the first hypothesis.

### Music at the beginning and end of the week

According to the second hypothesis, on Monday morning certain genres predominate in Moscow, while others dominate in St. Petersburg. Similarly, Friday evenings are dominated by different genres, depending on the city.

Save tables with data in two variables:
* in Moscow - in `moscow_general`;
* in St. Petersburg - in `spb_general`.

In [28]:
# getting the moscow_general table from those rows of the df table,
# for which the value in the 'city' column is 'Moscow'
moscow_general = df[df["city"] == "Moscow"]
moscow_general.head()

,user_id,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday


In [29]:
# getting the spb_general table from those rows of the df table,
# for which the value in the 'city' column is 'Saint-Petersburg'
spb_general = df[df["city"] == "Saint-Petersburg"]
spb_general.head()

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday


Let's create a function `genre_weekday()` with four parameters:
* table (dataframe) with data,
* day of the week,
* start timestamp in 'hh:mm' format,
* last timestamp in 'hh:mm' format.

The function should return information about the top 10 genres of those tracks that were listened to on the specified day, in the interval between two timestamps.

In [30]:
# Declaring the function genre_weekday() with parameters table, day, time1, time2,
# which returns information about the most popular genres on the specified day in
# given time:
# 1) the genre_df variable saves those rows of the transmitted dataframe table, for
# of which at the same time:
# - the value in the day column is equal to the value of the day argument
# - the value in the time column is greater than the value of the time1 argument
# - the value in the time column is less than the value of the time2 argument
# Use sequential filtering with boolean indexing.
# 2) group dataframe genre_df by genre column, take one of its
# columns and use the count() method to count the number of entries for each
# of present genres, write the resulting Series to a variable
#genre_df_count
# 3) sort genre_df_count in descending order of occurrence and save
# into the genre_df_sorted variable
# 4) return a Series of the first 10 genre_df_sorted values, these will be the top 10
# popular genres (on the specified day, at the specified time)
def genre_weekday(df , day, time1, time2):
    genre_df = df[(df["day"] ==day) & (df["time"] > time1) & (df["time"] <= time2)]
    
    genre_df_sorted  = genre_df.groupby("genre")["genre"].count().sort_values(ascending = False).head(10)
    
    
    return genre_df_sorted
    
        

Compare the results of the `genre_weekday()` function for Moscow and St. Petersburg on Monday morning (from 7:00 to 11:00) and Friday evening (from 17:00 to 23:00):

In [31]:
# function call for Monday morning in Moscow (instead of df — moscow_general table)
# time objects are strings and are compared as strings
# call example: genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')


genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [32]:

genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [33]:

genre_weekday(moscow_general, 'Friday', '18:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [34]:

genre_weekday(spb_general, 'Friday', '18:00', '23:00')

genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Findings**

If we compare the top 10 genres on Monday morning, we can draw the following conclusions:

1. In Moscow and St. Petersburg they listen to similar music. The only difference is that the Moscow rating includes the “world” genre, while the St. Petersburg rating includes jazz and classical.

2. There were so many missing values ​​in Moscow that the value `'unknown'` took tenth place among the most popular genres. This means that missing values ​​occupy a significant share in the data and threaten the reliability of the study.

Friday night does not change this picture. Some genres rise a little higher, others go down, but overall the top 10 stays the same.

Thus, the second hypothesis was only partially confirmed:
* Users listen to similar music at the beginning of the week and at the end.
* The difference between Moscow and St. Petersburg is not very pronounced. In Moscow, they listen to Russian popular music more often, in St. Petersburg - jazz.

However, gaps in the data cast doubt on this result. There are so many of them in Moscow that the top 10 ranking could look different if it were not for the lost genre data.

### Genre preferences in Moscow and St. Petersburg

Hypothesis: St. Petersburg is the capital of rap, the music of this genre is listened to more often than in Moscow. And Moscow is a city of contrasts, which, nevertheless, is dominated by pop music.

Let's group the `moscow_general` table by genre and count the listens of tracks of each genre using the `count()` method. Then sort the result in descending order and store it in the `moscow_genres` table.

In [ ]:
# in one line: grouping the moscow_general table by the 'genre' column,
# counting the number of 'genre' values ​​in this grouping using the count() method,
# sort the resulting Series in descending order and save to moscow_genres
moscow_genres = moscow_general.groupby("genre")["genre"].count().sort_values(ascending=False)


Let's display the first ten lines of `moscow_genres`:

In [36]:
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Now let's repeat the same for St. Petersburg.

Group the `spb_general` table by genre. Count the number of listens to tracks of each genre. Sort the result in descending order and store it in the `spb_genres` table:


In [37]:
# in one line: group spb_general table by 'genre' column,
# counting the number of 'genre' values ​​in this grouping using the count() method,
# sort the resulting Series in descending order and store it in spb_genres
spb_genres = spb_general.groupby("genre")["genre"].count().sort_values(ascending=False)

Print the first ten lines of `spb_genres` to the screen:

In [38]:
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Findings**

The hypothesis was partially confirmed:
* Pop music is the most popular genre in Moscow, as the hypothesis suggested. Moreover, in the top 10 genres there is a close genre - Russian popular music.
* Contrary to expectations, rap is equally popular in Moscow and St. Petersburg.

## CONCLUSION

We tested three hypotheses and found:

1. The day of the week has a different effect on the activity of users in Moscow and St. Petersburg.

The first hypothesis was fully confirmed.

2. Musical preferences do not change much during the week - be it Moscow or St. Petersburg. Small differences are noticeable at the beginning of the week, on Mondays:
* in Moscow they listen to music of the “world” genre,
* in St. Petersburg - jazz and classical music.

Thus, the second hypothesis was only partly confirmed. This result could have been different were it not for gaps in the data.

3. The tastes of users of Moscow and St. Petersburg have more in common than differences. Contrary to expectations, genre preferences in St. Petersburg resemble those in Moscow.

The third hypothesis was not confirmed. If there are differences in preferences, they are invisible to the bulk of users.

**In practice, studies contain tests of statistical hypotheses.**
From the data of one service, it is not always possible to draw a conclusion about all the inhabitants of the city.
Tests of statistical hypotheses will show how reliable they are, based on the available data.
You will learn more about hypothesis testing methods in the following topics.